In [16]:
import sqlite3
import requests
import time
import logging
import os

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def get_steam_page_info(app_id):
    url = f"https://store.steampowered.com/app/{app_id}/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Vérifier la présence de contenu généré par IA
        ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
        
        # Récupérer les tags
        tag_elements = soup.find_all('a', class_='app_tag')
        tags = [tag.text.strip() for tag in tag_elements][:4]  # Limiter à 4 tags
        
        # Récupérer le lien Twitter s'il existe
        twitter_link = soup.find('a', class_="ttip", attrs={'data-tooltip-text': lambda x: x and 'x.com/' in x})
        twitter_handle = None
        if twitter_link:
            twitter_url = twitter_link['data-tooltip-text']
            twitter_handle = '@' + twitter_url.split('/')[-1]
        
        return {
            'ai_generated': bool(ai_disclosure),
            'tags': tags,
            'twitter_handle': twitter_handle
        }
    except Exception as e:
        logging.error(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
        return None
    

content_descriptors = game_data.get('content_descriptors', {})
    if isinstance(content_descriptors, dict):
        descriptor_ids = content_descriptors.get('ids', [])
    elif isinstance(content_descriptors, list):
        descriptor_ids = content_descriptors
    else:
        descriptor_ids = []
    if 3 in descriptor_ids:
        return True

IndentationError: unexpected indent (2281960143.py, line 66)

In [ ]:
db_url = f"https://raw.githubusercontent.com/{os.getenv('PAT_GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
logging.info(f"URL de la base de données : {db_url}")

last_timestamp = 207817
total_games = 0
total_porn_games = 0
total_ai_games = 0

with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
    if download_db(db_url, temp_db.name):
        logging.info(f"Base de données téléchargée avec succès : {temp_db.name}")
        conn = connect_to_db(temp_db.name)
        new_entries = check_new_entries(conn, last_timestamp)
        logging.info(f"Nombre de nouvelles entrées trouvées : {len(new_entries)}")
        
        for steam_game_id, first_seen in new_entries:
            total_games += 1
            game_data = get_game_details(steam_game_id)
            if game_data:
                steam_page_info = get_steam_page_info(steam_game_id)
                if steam_page_info['ai_generated']:
                    total_ai_games += 1
                    if 3 in steam_page_info['content_descriptor'] :


    os.unlink(temp_db.name)        

print(f"\nRésumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.  ")
    


IndentationError: expected an indented block after 'if' statement on line 23 (3497198935.py, line 26)

In [ ]:
import sqlite3
import requests
import time
import logging
import os
import tempfile
from bs4 import BeautifulSoup
import re

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'

# Configuration du logging
logging.basicConfig(filename='count_games.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn, last_timestamp):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen > ?
        ORDER BY first_seen ASC
    """, (last_timestamp,))
    return cursor.fetchall()

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def get_steam_page_info(app_id):
    url = f"https://store.steampowered.com/app/{app_id}/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Vérifier la présence de contenu généré par IA
        ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
        
        return {
            'ai_generated': bool(ai_disclosure)
        }
    except Exception as e:
        logging.error(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
        return None

def has_id_3_in_content_descriptor(game_data):
    content_descriptors = game_data.get('content_descriptors', {})
    if isinstance(content_descriptors, dict):
        descriptor_ids = content_descriptors.get('ids', [])
    elif isinstance(content_descriptors, list):
        descriptor_ids = content_descriptors
    else:
        descriptor_ids = []
    return 3 in descriptor_ids

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('PAT_GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    logging.info(f"URL de la base de données : {db_url}")
    
    last_timestamp = 207817
    total_games = 0
    total_porn_games = 0
    total_ai_games = 0

    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            logging.info(f"Base de données téléchargée avec succès : {temp_db.name}")
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, last_timestamp)
            logging.info(f"Nombre de nouvelles entrées trouvées : {len(new_entries)}")
            
            for steam_game_id, first_seen in new_entries:
                total_games += 1
                game_data = get_game_details(steam_game_id)
                if game_data:
                    if has_id_3_in_content_descriptor(game_data):
                        total_porn_games += 1
                    
                    steam_page_info = get_steam_page_info(steam_game_id)
                    if steam_page_info and steam_page_info['ai_generated']:
                        total_ai_games += 1
            
            conn.close()
        
        os.unlink(temp_db.name)
    
    print(f"\nRésumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")
    logging.info(f"Résumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")

if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [ ]:
import sqlite3
import requests
import time
import logging
import os
import tempfile
from bs4 import BeautifulSoup
import re

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
MAX_GAMES_TO_PROCESS = 100

# Configuration du logging
logging.basicConfig(filename='count_games.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn, last_timestamp):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen > ?
        ORDER BY first_seen ASC
    """, (last_timestamp,))
    return cursor.fetchall()

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data[str(steam_game_id)]['success']:
                return data[str(steam_game_id)]['data']
    except requests.exceptions.RequestException as e:
        logging.error(f"Erreur lors de la requête pour le jeu {steam_game_id}: {e}")
    return None

def get_steam_page_info(app_id):
    url = f"https://store.steampowered.com/app/{app_id}/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
        
        return {
            'ai_generated': bool(ai_disclosure)
        }
    except Exception as e:
        logging.error(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
        return None


def has_id_3_in_content_descriptor(game_data):
    content_descriptors = game_data.get('content_descriptors', {})
    if isinstance(content_descriptors, dict):
        descriptor_ids = content_descriptors.get('ids', [])
    elif isinstance(content_descriptors, list):
        descriptor_ids = content_descriptors
    else:
        descriptor_ids = []
    return 3 in descriptor_ids

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('PAT_GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    logging.info(f"URL de la base de données : {db_url}")
    
    last_timestamp = 207817
    total_games = 0
    total_porn_games = 0
    total_ai_games = 0

    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            logging.info(f"Base de données téléchargée avec succès : {temp_db.name}")
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, last_timestamp)
            logging.info(f"Nombre de nouvelles entrées trouvées : {len(new_entries)}")
            
            start_time = time.time()
            for i, (steam_game_id, first_seen) in enumerate(new_entries, 1):
                if MAX_GAMES_TO_PROCESS and i > MAX_GAMES_TO_PROCESS:
                    break

                total_games += 1
                if i % 10 == 0:
                    elapsed_time = time.time() - start_time
                    avg_time_per_game = elapsed_time / i
                    estimated_total_time = avg_time_per_game * len(new_entries)
                    logging.info(f"Progression: {i}/{len(new_entries)} jeux traités. Temps estimé restant: {(estimated_total_time - elapsed_time)/60:.2f} minutes")
                    print(f"Progression: {i}/{len(new_entries)} jeux traités.")

                game_data = get_game_details(steam_game_id)
                if game_data:
                    if has_id_3_in_content_descriptor(game_data):
                        total_porn_games += 1
            
                    steam_page_info = get_steam_page_info(steam_game_id)
                    if steam_page_info and steam_page_info['ai_generated']:
                        total_ai_games += 1
        
                time.sleep(1) 
            
            conn.close()
        
        os.unlink(temp_db.name)
    
    print(f"\nRésumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")
    logging.info(f"Résumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")

if __name__ == "__main__":
    main()

Progression: 10/207817 jeux traités.
Progression: 20/207817 jeux traités.


KeyboardInterrupt: 

In [ ]:
import sqlite3
import requests
import time
import logging
import os
import tempfile
from bs4 import BeautifulSoup
import re

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
MIN_TIMESTAMP = 207817

# Configuration du logging
logging.basicConfig(filename='count_games.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn, min_timestamp):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen > ?
        ORDER BY first_seen ASC
    """, (min_timestamp,))
    return cursor.fetchall()

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data[str(steam_game_id)]['success']:
                return data[str(steam_game_id)]['data']
    except requests.exceptions.RequestException as e:
        logging.error(f"Erreur lors de la requête pour le jeu {steam_game_id}: {e}")
    return None

def get_steam_page_info(app_id):
    url = f"https://store.steampowered.com/app/{app_id}/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
        
        return {
            'ai_generated': bool(ai_disclosure)
        }
    except Exception as e:
        logging.error(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
        return None

def has_id_3_in_content_descriptor(game_data):
    content_descriptors = game_data.get('content_descriptors', {})
    if isinstance(content_descriptors, dict):
        descriptor_ids = content_descriptors.get('ids', [])
    elif isinstance(content_descriptors, list):
        descriptor_ids = content_descriptors
    else:
        descriptor_ids = []
    return 3 in descriptor_ids

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('PAT_GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    logging.info(f"URL de la base de données : {db_url}")
    
    total_games = 0
    total_porn_games = 0
    total_ai_games = 0

    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            logging.info(f"Base de données téléchargée avec succès : {temp_db.name}")
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, MIN_TIMESTAMP)
            logging.info(f"Nombre de nouvelles entrées trouvées : {len(new_entries)}")
            
            start_time = time.time()
            for i, (steam_game_id, first_seen) in enumerate(new_entries, 1):
                total_games += 1
                if i % 10 == 0:
                    elapsed_time = time.time() - start_time
                    avg_time_per_game = elapsed_time / i
                    estimated_total_time = avg_time_per_game * len(new_entries)
                    logging.info(f"Progression: {i}/{len(new_entries)} jeux traités. Temps estimé restant: {(estimated_total_time - elapsed_time)/60:.2f} minutes")
                    print(f"Progression: {i}/{len(new_entries)} jeux traités.")

                game_data = get_game_details(steam_game_id)
                if game_data:
                    if has_id_3_in_content_descriptor(game_data):
                        total_porn_games += 1
                    
                    steam_page_info = get_steam_page_info(steam_game_id)
                    if steam_page_info and steam_page_info['ai_generated']:
                        total_ai_games += 1
                
                time.sleep(1)  # Pause d'une seconde entre chaque requête
            
            conn.close()
        
        os.unlink(temp_db.name)
    
    print(f"\nRésumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")
    logging.info(f"Résumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")

if __name__ == "__main__":
    main()

Progression: 10/207817 jeux traités.
Progression: 20/207817 jeux traités.


KeyboardInterrupt: 

In [1]:
import sqlite3
import requests
import time
import logging
import os
import tempfile
from bs4 import BeautifulSoup
import re

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
MIN_TIMESTAMP = 207817

# Configuration du logging
logging.basicConfig(filename='count_games.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn, min_timestamp):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen > ?
        ORDER BY first_seen DESC
    """, (min_timestamp,))
    new_entries = []
    for row in cursor:
        new_entries.append(row)
    new_entries.reverse()  # Pour avoir les entrées dans l'ordre chronologique
    return new_entries

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            if data[str(steam_game_id)]['success']:
                return data[str(steam_game_id)]['data']
    except requests.exceptions.RequestException as e:
        logging.error(f"Erreur lors de la requête pour le jeu {steam_game_id}: {e}")
    return None

def get_steam_page_info(app_id):
    url = f"https://store.steampowered.com/app/{app_id}/"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        ai_disclosure = soup.find(string=re.compile("AI GENERATED CONTENT DISCLOSURE", re.IGNORECASE))
        
        return {
            'ai_generated': bool(ai_disclosure)
        }
    except Exception as e:
        logging.error(f"Erreur lors du scraping pour le jeu {app_id}: {e}")
        return None

def has_id_3_in_content_descriptor(game_data):
    content_descriptors = game_data.get('content_descriptors', {})
    if isinstance(content_descriptors, dict):
        descriptor_ids = content_descriptors.get('ids', [])
    elif isinstance(content_descriptors, list):
        descriptor_ids = content_descriptors
    else:
        descriptor_ids = []
    return 3 in descriptor_ids

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('PAT_GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    logging.info(f"URL de la base de données : {db_url}")
    
    total_games = 0
    total_porn_games = 0
    total_ai_games = 0

    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            logging.info(f"Base de données téléchargée avec succès : {temp_db.name}")
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, MIN_TIMESTAMP)
            logging.info(f"Nombre de nouvelles entrées trouvées : {len(new_entries)}")
            
            start_time = time.time()
            for i, (steam_game_id, first_seen) in enumerate(new_entries, 1):
                total_games += 1
                if i % 10 == 0:
                    elapsed_time = time.time() - start_time
                    avg_time_per_game = elapsed_time / i
                    estimated_total_time = avg_time_per_game * len(new_entries)
                    logging.info(f"Progression: {i}/{len(new_entries)} jeux traités. Temps estimé restant: {(estimated_total_time - elapsed_time)/60:.2f} minutes")
                    print(f"Progression: {i}/{len(new_entries)} jeux traités.")

                game_data = get_game_details(steam_game_id)
                if game_data:
                    if has_id_3_in_content_descriptor(game_data):
                        total_porn_games += 1
                    
                    steam_page_info = get_steam_page_info(steam_game_id)
                    if steam_page_info and steam_page_info['ai_generated']:
                        total_ai_games += 1
                
                time.sleep(1)  # Pause d'une seconde entre chaque requête
            
            conn.close()
        
        os.unlink(temp_db.name)
    
    print(f"\nRésumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")
    logging.info(f"Résumé : {total_games} nouvelles pages steam. {total_porn_games} jeux porno et {total_ai_games} jeux AI.")

if __name__ == "__main__":
    main()

Progression: 10/207817 jeux traités.
Progression: 20/207817 jeux traités.
Progression: 30/207817 jeux traités.
Progression: 40/207817 jeux traités.
Progression: 50/207817 jeux traités.
Progression: 60/207817 jeux traités.
Progression: 70/207817 jeux traités.
Progression: 80/207817 jeux traités.
Progression: 90/207817 jeux traités.
Progression: 100/207817 jeux traités.
Progression: 110/207817 jeux traités.
Progression: 120/207817 jeux traités.
Progression: 130/207817 jeux traités.
Progression: 140/207817 jeux traités.
Progression: 150/207817 jeux traités.
Progression: 160/207817 jeux traités.
Progression: 170/207817 jeux traités.
Progression: 180/207817 jeux traités.
Progression: 190/207817 jeux traités.
Progression: 200/207817 jeux traités.
Progression: 210/207817 jeux traités.
Progression: 220/207817 jeux traités.
Progression: 230/207817 jeux traités.
Progression: 240/207817 jeux traités.
Progression: 250/207817 jeux traités.
Progression: 260/207817 jeux traités.
Progression: 270/2078

KeyboardInterrupt: 